In [1]:
# Re-import necessary libraries due to environment reset
import pandas as pd

# Reload the dataset
df = pd.read_csv("preprocessed_dataset_0510.csv", parse_dates=["timestamp"])

# Preview structure and first few rows
df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391095 entries, 0 to 391094
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   timestamp        391095 non-null  datetime64[ns, UTC]
 1   pod              391095 non-null  object             
 2   container        391095 non-null  object             
 3   cpu_usage        391095 non-null  float64            
 4   label            391095 non-null  int64              
 5   memory_usage     391095 non-null  float64            
 6   net_receive      391095 non-null  float64            
 7   net_transmit     391095 non-null  float64            
 8   fs_reads_bytes   391095 non-null  float64            
 9   fs_writes_bytes  391095 non-null  float64            
 10  restarts         391095 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(7), int64(1), object(2)
memory usage: 32.8+ MB


(None,
                   timestamp                          pod  container  \
 0 2025-09-22 23:00:00+00:00  anomaly-load-29308860-54nq2         k6   
 1 2025-09-22 23:00:00+00:00  anomaly-load-29308920-bckkd         k6   
 2 2025-09-22 23:00:00+00:00  anomaly-load-29309220-6hl4k         k6   
 3 2025-09-22 23:00:00+00:00        cart-5d9c9599cb-89l9k       cart   
 4 2025-09-22 23:00:00+00:00   catalogue-6878884699-n4xl5  catalogue   
 
    cpu_usage  label  memory_usage  net_receive  net_transmit  fs_reads_bytes  \
 0   0.000000      0           0.0          0.0           0.0             0.0   
 1   0.000000      0           0.0          0.0           0.0             0.0   
 2   0.000000      0           0.0          0.0           0.0             0.0   
 3   0.000550      0    45854720.0          0.0           0.0             0.0   
 4   0.000688      0    52903936.0          0.0           0.0             0.0   
 
    fs_writes_bytes  restarts  
 0              0.0       0.0  
 1     

# Time sequencing for LSTM part starts here.

In [2]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Limit dataset to key services for faster LSTM prep
key_containers = ['cart', 'catalogue', 'web']
df_filtered = df[df['container'].isin(key_containers)]

# Sort and reduce to last 5000 entries per pod
df_sequences = (
    df_filtered.sort_values(by=["pod", "timestamp"])
    .groupby("pod")
    .tail(5000)
    .reset_index(drop=True)
)

# Normalize features
FEATURE_COLS = [
    "cpu_usage", "memory_usage", "net_receive", "net_transmit",
    "fs_reads_bytes", "fs_writes_bytes", "restarts"
]
WINDOW_SIZE = 30
scaler = MinMaxScaler()
df_sequences[FEATURE_COLS] = scaler.fit_transform(df_sequences[FEATURE_COLS])

# Build LSTM sequences
X, y = [], []
for pod_name, pod_df in df_sequences.groupby("pod"):
    pod_df = pod_df.reset_index(drop=True)
    for i in range(len(pod_df) - WINDOW_SIZE + 1):
        window = pod_df.loc[i:i+WINDOW_SIZE-1, FEATURE_COLS].values
        label = pod_df.loc[i+WINDOW_SIZE-1, "label"]
        X.append(window)
        y.append(label)

X = np.array(X)
y = np.array(y)

X.shape, y.shape

((14913, 30, 7), (14913,))

In [3]:
pip install tensorflow==2.15

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 55.5 MB/s  0:00:05m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 172.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 134.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 85.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 121.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 259.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 186.9 MB/s  0:00:00
  Attempting uninstall: wrapt━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  1/24 [flatbuffers]
    Found existing installation: wrapt 2.0.0━━━━━━━━━━━━━━━━━━  1/24 [flatbuffers]
    Uninstalling wrapt-2.0.0:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  1/24 [flatbuffers]
      Successfully uninstalled wrapt-2.0.0━━━━━━━━━━━━━━━━━━━━  1/24 [flatbuffers]
  Attempting uninstall: protobufm━━━━━━━━━━━━━━━━━━━━━━━━━━━━  7/24 [pyasn1-modules]mator]esystem]
    Found existing inst

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Assuming X and y already exist in memory
# If not, load them from files: X = np.load('X.npy'); y = np.load('y.npy')

# Train-test split (70-30)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Define the LSTM model
model = Sequential([
    LSTM(64, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Evaluation
y_pred = (model.predict(X_test) > 0.5).astype("int32")

print("📊 Classification Report:")
print(classification_report(y_test, y_pred))

print("📉 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Save model
model.save("lstm_model.h5")
print("✅ Model saved as lstm_model.h5")

2025-11-18 13:01:13.601170: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-18 13:01:13.601209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-18 13:01:13.602363: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-18 13:01:13.609814: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-18 13:01:14.667758: W tensorflow/comp

Epoch 1/20
131/131 [==============================] - 7s 32ms/step - loss: 0.1293 - accuracy: 0.9780 - val_loss: 0.0850 - val_accuracy: 0.9832
Epoch 2/20
131/131 [==============================] - 3s 26ms/step - loss: 0.0800 - accuracy: 0.9850 - val_loss: 0.0850 - val_accuracy: 0.9832
Epoch 3/20
131/131 [==============================] - 3s 27ms/step - loss: 0.0793 - accuracy: 0.9850 - val_loss: 0.0842 - val_accuracy: 0.9832
Epoch 4/20
131/131 [==============================] - 3s 26ms/step - loss: 0.0783 - accuracy: 0.9850 - val_loss: 0.0829 - val_accuracy: 0.9832
Epoch 5/20
131/131 [==============================] - 3s 26ms/step - loss: 0.0771 - accuracy: 0.9850 - val_loss: 0.0797 - val_accuracy: 0.9832
Epoch 6/20
131/131 [==============================] - 3s 26ms/step - loss: 0.0720 - accuracy: 0.9850 - val_loss: 0.0721 - val_accuracy: 0.9832
Epoch 7/20
131/131 [==============================] - 3s 26ms/step - loss: 0.0660 - accuracy: 0.9850 - val_loss: 0.0671 - val_accuracy: 0.9832

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
